In [1]:
# | default_exp gsc_client


In [2]:
# | export
import pickle
from datetime import datetime, timedelta
from pathlib import Path
from fastcore.basics import store_attr

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request as GoogleRequest

In [3]:
# | export
class GSCAuth:
    """Google Search Console Authentication Handler"""

    def __init__(
        self,
        secrets_file: str = "./client_secrets.json",
        token_file: str = "./token.pickle",
    ):
        store_attr()
        self.token_file = Path(token_file)
        self.credentials = self._load_credentials()

    def _load_credentials(self) -> Credentials | None:
        if self.token_file.exists():
            with open(self.token_file, "rb") as token:
                return pickle.load(token)
        return None

    def _save_credentials(self, credentials: Credentials):
        with open(self.token_file, "wb") as token:
            pickle.dump(credentials, token)

    def get_credentials(self) -> Credentials | None:
        if not self.credentials:
            return None
        if self.credentials.expired:
            self.credentials.refresh(GoogleRequest())
            self._save_credentials(self.credentials)
        return self.credentials

    def authenticate(self):
        flow = InstalledAppFlow.from_client_secrets_file(
            self.secrets_file,
            scopes=[
                "https://www.googleapis.com/auth/webmasters.readonly",
                "https://www.googleapis.com/auth/webmasters",
            ],
        )
        self.credentials = flow.run_local_server(port=0)
        self._save_credentials(self.credentials)


In [4]:
# | hide
auth = GSCAuth(secrets_file="client_secrets.json")
auth.authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=379261216946-18ru6cgt2sha1grk8vr0tu81kv1p0jir.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54605%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters&state=UeS8PUmMpWjJbN7NY0kuPZE269unK4&access_type=offline


In [5]:
# | export
def get_date_range(
    range_type: str = "today",
    days: int | None = None,
    months: int | None = None,
    start_date: str | None = None,
    end_date: str | None = None,
) -> tuple[str, str]:
    """Generate date range for GSC queries (accounts for 3-day delay)"""
    latest = datetime.now() - timedelta(days=3)

    match range_type:
        case "today":
            d = latest.strftime("%Y-%m-%d")
            return d, d
        case "last_days" if days:
            start = latest - timedelta(days=days)
            return start.strftime("%Y-%m-%d"), latest.strftime("%Y-%m-%d")
        case "last_months" if months:
            start = latest - timedelta(days=30 * months)
            return start.strftime("%Y-%m-%d"), latest.strftime("%Y-%m-%d")
        case "custom" if start_date and end_date:
            s = datetime.strptime(start_date, "%Y-%m-%d")
            e = min(datetime.strptime(end_date, "%Y-%m-%d"), latest)
            return s.strftime("%Y-%m-%d"), e.strftime("%Y-%m-%d")
        case _:
            raise ValueError("Invalid date range parameters")

In [6]:
# | export
def fetch_gsc_data(
    auth: GSCAuth,
    site_url: str,
    start_date: str,
    end_date: str,
    dimensions: list[str] | None = None,
    row_limit: int = 25000,
) -> list[dict]:
    """Fetch data from Google Search Console"""
    if dimensions is None:
        dimensions = ["query", "page", "country", "device"]

    service = build("searchconsole", "v1", credentials=auth.get_credentials())
    request = {
        "startDate": start_date,
        "endDate": end_date,
        "dimensions": dimensions,
        "rowLimit": row_limit,
    }
    return (
        service.searchanalytics()
        .query(siteUrl=site_url, body=request)
        .execute()
        .get("rows", [])
    )


In [7]:
# | export
def get_verified_sites(auth: GSCAuth) -> list[dict]:
    """Get list of verified sites"""
    service = build("searchconsole", "v1", credentials=auth.get_credentials())
    sites_list = service.sites().list().execute()
    return [
        {"site_url": site["siteUrl"], "permission_level": site["permissionLevel"]}
        for site in sites_list.get("siteEntry", [])
        if site["permissionLevel"] != "siteUnverifiedUser"
    ]


In [8]:
# | hide

sites = get_verified_sites(auth)
sites


[{'site_url': 'sc-domain:kamcalorie.com', 'permission_level': 'siteFullUser'},
 {'site_url': 'sc-domain:smaagarden.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:protienai.com', 'permission_level': 'siteFullUser'},
 {'site_url': 'sc-domain:awazly.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:tanorfix.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:azlzone.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:kareemai.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:alainclean.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:gpuvec.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:leakdetectionkw.com',
  'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:shelid.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:emdadelgaz.com', 'permission_level': 'siteOwner'}]

In [9]:
# | hide
from pprint import pprint

start, end = get_date_range("last_days", 1)
data = fetch_gsc_data(auth, "sc-domain:kareemai.com", start, end)
pprint(data)


[{'clicks': 1,
  'ctr': 1,
  'impressions': 1,
  'keys': ['freebuds 7i review',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'egy',
           'MOBILE'],
  'position': 1},
 {'clicks': 1,
  'ctr': 0.5,
  'impressions': 2,
  'keys': ['huawei 7i earbuds',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'usa',
           'MOBILE'],
  'position': 4},
 {'clicks': 1,
  'ctr': 1,
  'impressions': 1,
  'keys': ['huawei 7i freebuds',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'gbr',
           'MOBILE'],
  'position': 5},
 {'clicks': 1,
  'ctr': 1,
  'impressions': 1,
  'keys': ['huawei 7i review',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'fin',
           'MOBILE'],
  'position': 1},
 {'clicks': 1,
  'ctr': 0.1,
  'impressions': 10,
  'keys': ['huawei freebuds 7i',
  